In [1]:
import os
import warnings
import sys

In [2]:
pip install mlflow


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [4]:
data_path = "wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2103,6.0,0.27,0.15,1.5,0.056,35.0,128.0,0.99360,3.12,0.45,8.8,5
3034,6.9,0.25,0.47,8.4,0.042,36.0,156.0,0.99604,3.15,0.55,9.4,6
4758,6.5,0.20,0.35,3.9,0.040,27.0,140.0,0.99102,2.98,0.53,11.8,6
4844,4.8,0.29,0.23,1.1,0.044,38.0,180.0,0.98924,3.28,0.34,11.9,6
648,6.0,0.32,0.12,5.9,0.041,34.0,190.0,0.99440,3.16,0.72,10.0,5
209,6.1,0.40,0.31,0.9,0.048,23.0,170.0,0.99300,3.22,0.77,9.5,6
2611,6.5,0.30,0.39,7.8,0.038,61.0,219.0,0.99590,3.19,0.50,9.4,5
4508,5.8,0.26,0.30,2.6,0.034,75.0,129.0,0.99020,3.20,0.38,11.5,4
2950,6.0,0.28,0.25,1.8,0.042,8.0,108.0,0.99290,3.08,0.55,9.0,5
1005,6.6,0.18,0.28,3.3,0.044,18.0,91.0,0.99300,3.42,0.64,10.8,6


In [10]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://127.0.0.1:5000/" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [11]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000/'

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import warnings

def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def load_data(data_path):
    data = pd.read_csv(data_path)
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data, test_size=0.25, random_state=42)
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y

def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Use the absolute file path
    data_path = "/home/ashutosh/my_project/Assign3.2/Mlflow/wine-quality.csv"

    train_x, train_y, test_x, test_y = load_data(data_path)

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(lr, "model")

if __name__ == "__main__":
    train(0.5, 0.5)


Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.821195103195106
  MAE: 0.6338934920844772
  R2: 0.1313572669551294
Save to: mlflow-artifacts:/0/8d7554e676e44d26a1b0c16159b88421/artifacts


In [14]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7854974519820409
  MAE: 0.6171230074647535
  R2: 0.20523625056452055
Save to: mlflow-artifacts:/0/1d319fef0e89410cabdcdda7b5a9d9e4/artifacts


In [15]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7801538925222868
  MAE: 0.6142807154069012
  R2: 0.21601266265188934
Save to: mlflow-artifacts:/0/b65e1cf2bd964fe2a7c7403ffc07b24d/artifacts
